# A 10-minute Buy-and-Hold template

This simple template provides basic instructions for helping you within 10 minutes to develop and submit a buy-and-hold strategy on liquid assets.

In [1]:
# import basic libraries for manipulating data:

import xarray as xr
import numpy as np
import pandas as pd

# import quantnet libraries:

from qnt.data import *
from qnt.stepper import *
from qnt.stats import *
from qnt.graph import *

# display function
from IPython.display import display

In [2]:
# load all available asset names:

assets = load_assets(min_date="2015-01-01")

assets_names = [x["id"] for x in assets]

# display sample:

for i in assets_names[:10]:
    print(i)

AMEX:LNG
AMEX:NG
AMEX:NGD
AMEX:NML
AMEX:NOG
AMEX:SVM
AMEX:VHC
NASDAQ:AABA
NASDAQ:AAL
NASDAQ:AAOI


In [3]:
# load all data: it is possible to set a max_date in order to test the system on unseen data,
# however submission requires not using any max_date:

data = load_data(min_date="2015-01-01",
                 #max_date="2019-02-28",
                 dims=("time", "field", "asset"),
                 assets=assets_names)

fetched chunk 1/11 0s
fetched chunk 2/11 1s
fetched chunk 3/11 1s
fetched chunk 4/11 1s
fetched chunk 5/11 1s
fetched chunk 6/11 1s
fetched chunk 7/11 2s
fetched chunk 8/11 2s
fetched chunk 9/11 2s
fetched chunk 10/11 2s
fetched chunk 11/11 3s
Data loaded 3s


In [4]:
# display number of assets:

print("Assets: " + str(len(data["asset"])))

# display number of trading days:

print("Trading days: " + str(len(data["time"])))

# display number of fields at disposal:

print("Fields: " + str(len(data["field"])))

Assets: 2085
Trading days: 1050
Fields: 9


In [5]:
# display data sample using coords method from xarray: price data (open, low, high, close),
# volume (vol, number of traded contracts), information about splits and dividends (read more
# in our Strategy Buy and Hold template) and liquidity tag (more on liquidity later):

for i in data.coords["asset"][:10]:
    print(i.values.tolist())
    display(data.loc[:,:,i].to_pandas().head())
    print("")

AMEX:LNG


field,open,low,high,close,vol,divs,split,split_cumprod,is_liquid
time,,,,,,,,,
2019-03-06,33.35,32.6600,33.3800,32.780,2953822.0,0.0,1.0,0.5,1.0
2019-03-05,32.72,32.6050,33.3200,33.265,4332486.0,0.0,1.0,0.5,1.0
2019-03-04,33.50,32.1750,33.5500,32.500,11102868.0,0.0,1.0,0.5,1.0
2019-03-01,32.21,31.3255,32.4950,32.000,5129626.0,0.0,1.0,0.5,1.0
2019-02-28,32.75,32.0250,32.9363,32.225,5820800.0,0.0,1.0,0.5,1.0



AMEX:NG


field,open,low,high,close,vol,divs,split,split_cumprod,is_liquid
time,,,,,,,,,
2019-03-06,3.93,3.850,3.96,3.89,1321354.0,0.0,1.0,1.0,0.0
2019-03-05,3.95,3.910,3.97,3.96,979352.0,0.0,1.0,1.0,0.0
2019-03-04,3.84,3.840,3.98,3.95,1212712.0,0.0,1.0,1.0,0.0
2019-03-01,3.96,3.865,3.98,3.93,1817224.0,0.0,1.0,1.0,0.0
2019-02-28,3.91,3.840,4.00,3.97,1807903.0,0.0,1.0,1.0,0.0



AMEX:NGD


...

field,open,low,high,close,vol,divs,split,split_cumprod,is_liquid
time,,,,,,,,,
2019-03-06,1210.88,1191.52,1210.88,1196.64,398764.6250,0.0,1.0,16.0,1.0
2019-03-05,1215.36,1201.28,1220.16,1201.92,439304.3125,0.0,1.0,16.0,1.0
2019-03-04,1211.20,1199.20,1218.88,1216.16,352303.7500,0.0,1.0,16.0,1.0
2019-03-01,1200.00,1194.56,1210.56,1199.36,314567.7500,0.0,1.0,16.0,1.0
2019-02-28,1189.92,1178.40,1199.20,1193.44,699154.3750,0.0,1.0,16.0,1.0



NASDAQ:AAL


field,open,low,high,close,vol,divs,split,split_cumprod,is_liquid
time,,,,,,,,,
2019-03-06,33.22,32.5150,33.415,32.76,6402192.0,0.0,1.0,1.0,1.0
2019-03-05,34.03,33.1700,34.380,33.23,7347512.0,0.0,1.0,1.0,1.0
2019-03-04,34.76,33.5000,34.890,33.72,6998090.0,0.0,1.0,1.0,1.0
2019-03-01,35.58,34.3700,35.960,34.64,8654044.0,0.0,1.0,1.0,1.0
2019-02-28,35.63,35.1699,35.890,35.63,7873833.0,0.0,1.0,1.0,1.0



NASDAQ:AAOI


field,open,low,high,close,vol,divs,split,split_cumprod,is_liquid
time,,,,,,,,,
2019-03-06,13.58,13.1100,13.7100,13.30,883423.0,0.0,1.0,1.0,0.0
2019-03-05,13.95,13.4599,13.9700,13.64,839353.0,0.0,1.0,1.0,0.0
2019-03-04,13.39,13.1100,14.1200,13.97,1758220.0,0.0,1.0,1.0,0.0
2019-03-01,13.63,13.2300,13.7892,13.24,3147934.0,0.0,1.0,1.0,0.0
2019-02-28,14.89,13.4900,14.9500,13.50,5093986.0,0.0,1.0,1.0,0.0


In [6]:
# visualize price evolution of a selected asset:

prices_amzn = data.loc[:, "close", "NASDAQ:AMZN"].to_series()
make_plot(prices_amzn.index, prices_amzn, name="AMZN")

In [7]:
# submissions have to trade liquid assets at all times. We provide a liquidity tag
# which takes values:

# 0:   illiquid stock;
# 1:   liquid stock;
# Nan: missing data (for example, an asset was not yet listed on the exchange or it has been delisted).

data.loc[:, "is_liquid"].to_pandas()

asset,AMEX:LNG,AMEX:NG,AMEX:NGD,AMEX:NML,AMEX:NOG,AMEX:SVM,AMEX:VHC,NASDAQ:AABA,NASDAQ:AAL,NASDAQ:AAOI,...,NYSE:YEXT,NYSE:YPF,NYSE:YRD,NYSE:YUM,NYSE:YUMC,NYSE:ZAYO,NYSE:ZBH,NYSE:ZEN,NYSE:ZTO,NYSE:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2019-03-06,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-03-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-03-04,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-03-01,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-02-28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-02-27,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-02-26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-02-25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2019-02-22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [8]:
# we define a simple buy-and-hold strategy on liquid assets allocating equal fractions
# of capital to all liquid assets at a given point in time:

is_liquid = data.loc[:,"is_liquid"]

liquid_sum = is_liquid.sum("asset")

# check and display number of liquid assets:

liquid_sum.to_pandas()

time
2019-03-06    789.0
2019-03-05    796.0
2019-03-04    800.0
2019-03-01    797.0
2019-02-28    796.0
2019-02-27    795.0
2019-02-26    797.0
2019-02-25    798.0
2019-02-22    796.0
2019-02-21    797.0
2019-02-20    804.0
2019-02-19    805.0
2019-02-15    804.0
2019-02-14    806.0
2019-02-13    809.0
2019-02-12    803.0
2019-02-11    800.0
2019-02-08    799.0
2019-02-07    801.0
2019-02-06    761.0
2019-02-05    764.0
2019-02-04    765.0
2019-02-01    767.0
2019-01-31    767.0
2019-01-30    767.0
2019-01-29    768.0
2019-01-28    771.0
2019-01-25    773.0
2019-01-24    773.0
2019-01-23    778.0
              ...  
2015-02-13    659.0
2015-02-12    642.0
2015-02-11    624.0
2015-02-10    614.0
2015-02-09    547.0
2015-02-06    467.0
2015-02-05    467.0
2015-02-04    467.0
2015-02-03    467.0
2015-02-02    467.0
2015-01-30    467.0
2015-01-29    467.0
2015-01-28    467.0
2015-01-27    467.0
2015-01-26    468.0
2015-01-23    468.0
2015-01-22    468.0
2015-01-21    468.0
2015-01-20    4

In [9]:
# define equal weights for going long:

weights = is_liquid/liquid_sum

# take care of corner case where liquid_sum=0:

weights = weights.fillna(0)

weights.to_pandas()

asset,AMEX:LNG,AMEX:NG,AMEX:NGD,AMEX:NML,AMEX:NOG,AMEX:SVM,AMEX:VHC,NASDAQ:AABA,NASDAQ:AAL,NASDAQ:AAOI,...,NYSE:YEXT,NYSE:YPF,NYSE:YRD,NYSE:YUM,NYSE:YUMC,NYSE:ZAYO,NYSE:ZBH,NYSE:ZEN,NYSE:ZTO,NYSE:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2019-03-06,0.001267,0.0,0.0,0.0,0.0,0.0,0.0,0.001267,0.001267,0.0,...,0.0,0.0,0.0,0.001267,0.001267,0.001267,0.001267,0.001267,0.0,0.001267
2019-03-05,0.001256,0.0,0.0,0.0,0.0,0.0,0.0,0.001256,0.001256,0.0,...,0.0,0.0,0.0,0.001256,0.001256,0.001256,0.001256,0.001256,0.0,0.001256
2019-03-04,0.001250,0.0,0.0,0.0,0.0,0.0,0.0,0.001250,0.001250,0.0,...,0.0,0.0,0.0,0.001250,0.001250,0.001250,0.001250,0.001250,0.0,0.001250
2019-03-01,0.001255,0.0,0.0,0.0,0.0,0.0,0.0,0.001255,0.001255,0.0,...,0.0,0.0,0.0,0.001255,0.001255,0.001255,0.001255,0.001255,0.0,0.001255
2019-02-28,0.001256,0.0,0.0,0.0,0.0,0.0,0.0,0.001256,0.001256,0.0,...,0.0,0.0,0.0,0.001256,0.001256,0.001256,0.001256,0.001256,0.0,0.001256
2019-02-27,0.001258,0.0,0.0,0.0,0.0,0.0,0.0,0.001258,0.001258,0.0,...,0.0,0.0,0.0,0.001258,0.001258,0.001258,0.001258,0.001258,0.0,0.001258
2019-02-26,0.001255,0.0,0.0,0.0,0.0,0.0,0.0,0.001255,0.001255,0.0,...,0.0,0.0,0.0,0.001255,0.001255,0.001255,0.001255,0.001255,0.0,0.001255
2019-02-25,0.001253,0.0,0.0,0.0,0.0,0.0,0.0,0.001253,0.001253,0.0,...,0.0,0.0,0.0,0.001253,0.001253,0.001253,0.001253,0.001253,0.0,0.001253
2019-02-22,0.001256,0.0,0.0,0.0,0.0,0.0,0.0,0.001256,0.001256,0.0,...,0.0,0.0,0.0,0.001256,0.001256,0.001256,0.001256,0.001256,0.0,0.001256


In [10]:
# check that we are fully invested and sum of weights=1:

weights.sum("asset").to_pandas()

time
2019-03-06    1.0
2019-03-05    1.0
2019-03-04    1.0
2019-03-01    1.0
2019-02-28    1.0
2019-02-27    1.0
2019-02-26    1.0
2019-02-25    1.0
2019-02-22    1.0
2019-02-21    1.0
2019-02-20    1.0
2019-02-19    1.0
2019-02-15    1.0
2019-02-14    1.0
2019-02-13    1.0
2019-02-12    1.0
2019-02-11    1.0
2019-02-08    1.0
2019-02-07    1.0
2019-02-06    1.0
2019-02-05    1.0
2019-02-04    1.0
2019-02-01    1.0
2019-01-31    1.0
2019-01-30    1.0
2019-01-29    1.0
2019-01-28    1.0
2019-01-25    1.0
2019-01-24    1.0
2019-01-23    1.0
             ... 
2015-02-13    1.0
2015-02-12    1.0
2015-02-11    1.0
2015-02-10    1.0
2015-02-09    1.0
2015-02-06    1.0
2015-02-05    1.0
2015-02-04    1.0
2015-02-03    1.0
2015-02-02    1.0
2015-01-30    1.0
2015-01-29    1.0
2015-01-28    1.0
2015-01-27    1.0
2015-01-26    1.0
2015-01-23    1.0
2015-01-22    1.0
2015-01-21    1.0
2015-01-20    1.0
2015-01-16    1.0
2015-01-15    1.0
2015-01-14    1.0
2015-01-13    1.0
2015-01-12    1.0
2015-

In [11]:
def step(data):
    """This function summarizes the previous strategy evaluating positions at
    any time point.

    Input:  data, an xarray with dims : field, time, asset

    Output: weights, an xarray with fractions which will be allocated to all
            assets on following day.
    """

    data_point = data[0]

    is_liquid = data_point.loc["is_liquid"]
    liquid_sum = is_liquid.sum("asset")
    weights = is_liquid/liquid_sum
    weights = weights.fillna(0)
    
    return weights

In [12]:
# a warm-up period:

init_data_length = 20

# call test_strategy: it will tell you if strategy is acceptable for submission

output = test_strategy(data, step=step, init_data_length=init_data_length)

Testing started...
Testing complete 4.106403827667236s


In [13]:
# mandatory step for submitting, write positions to file:

write_output(output)

# if all steps have been completed, you can click on the submit button!

write output: /home/username/fractions.nc.gz


In [14]:
# now we take a look at the generated positions:

output.to_pandas().tail()

asset,AMEX:LNG,AMEX:NG,AMEX:NGD,AMEX:NML,AMEX:NOG,AMEX:SVM,AMEX:VHC,NASDAQ:AABA,NASDAQ:AAL,NASDAQ:AAOI,...,NYSE:YEXT,NYSE:YPF,NYSE:YRD,NYSE:YUM,NYSE:YUMC,NYSE:ZAYO,NYSE:ZBH,NYSE:ZEN,NYSE:ZTO,NYSE:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2019-02-28,0.001256,0.0,0.0,0.0,0.0,0.0,0.0,0.001256,0.001256,0.0,...,0.0,0.0,0.0,0.001256,0.001256,0.001256,0.001256,0.001256,0.0,0.001256
2019-03-01,0.001255,0.0,0.0,0.0,0.0,0.0,0.0,0.001255,0.001255,0.0,...,0.0,0.0,0.0,0.001255,0.001255,0.001255,0.001255,0.001255,0.0,0.001255
2019-03-04,0.001250,0.0,0.0,0.0,0.0,0.0,0.0,0.001250,0.001250,0.0,...,0.0,0.0,0.0,0.001250,0.001250,0.001250,0.001250,0.001250,0.0,0.001250
2019-03-05,0.001256,0.0,0.0,0.0,0.0,0.0,0.0,0.001256,0.001256,0.0,...,0.0,0.0,0.0,0.001256,0.001256,0.001256,0.001256,0.001256,0.0,0.001256
2019-03-06,0.001267,0.0,0.0,0.0,0.0,0.0,0.0,0.001267,0.001267,0.0,...,0.0,0.0,0.0,0.001267,0.001267,0.001267,0.001267,0.001267,0.0,0.001267


In [15]:
# we visualize positions for a selected asset:

aapl_frac = output.to_pandas()["NASDAQ:AAPL"]

make_plot(aapl_frac.index, aapl_frac, name= "aapl")

In [16]:
# calculate statitistics on a rolling basis, punishing transactions with slippage (read more in our
# Strategy Buy and Hold template); volatility, sharpe ratio and mean return are computed on a rolling
# period of 1 year

stat = calc_stat(data, output, slippage_factor=0.05)

stat.to_pandas().tail()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,trading_duration
time,,,,,,,,,,
2019-02-28,1.325296,-0.003205,0.163343,-0.040016,0.236386,0.205059,0.033495,1.0,1217.0,74.643177
2019-03-01,1.334466,0.006919,0.163082,-0.033374,0.236386,0.321378,0.052411,1.0,1217.0,74.626499
2019-03-04,1.326191,-0.006200,0.162994,-0.039367,0.236386,0.334081,0.054453,1.0,1218.0,74.567835
2019-03-05,1.324880,-0.000989,0.162796,-0.040317,0.236386,0.274314,0.044657,1.0,1218.0,74.678939
2019-03-06,1.311344,-0.010217,0.162801,-0.050122,0.236386,0.142713,0.023234,1.0,1218.0,74.715927


In [17]:
def print_stat(stat):
    """Prints selected statistical key indicators:
    -- the Sharpe ratio of the strategy;
    -- the mean profit;
    -- the volatility;
    -- the maximum drawdown.
    Note that Sharpe ratio, mean profit and volatility apply to max simulation period.
    """

    days_num = len(stat.coords["time"])

    sharpe_ratio = calc_sharpe_ratio_annualized(stat.loc[:, "relative_return"],
                    max_periods=days_num, min_periods=days_num).to_pandas().values[-1]

    profit = (calc_mean_return_annualized(stat.loc[:, "relative_return"],
                    max_periods=days_num, min_periods=days_num).to_pandas().values[-1])*100.0

    volatility = (calc_volatility_annualized(stat.loc[:, "relative_return"], 
                    max_periods=days_num, min_periods=days_num).to_pandas().values[-1])*100.0

    max_ddown = (calc_max_drawdown(calc_underwater(stat.loc[:, "equity"])).to_pandas().values[-1])*100.0

    print("Sharpe Ratio     : ", "{0:.3f}".format(sharpe_ratio))
    print("Profit [%]       : ", "{0:.3f}".format(profit))
    print("Volatility [%]   : ", "{0:.3f}".format(volatility))
    print("Max Drawdown [%] : ", "{0:.3f}".format(max_ddown))

print_stat(stat)

Sharpe Ratio     :  0.462
Profit [%]       :  6.863
Volatility [%]   :  14.858
Max Drawdown [%] :  23.639


In [18]:
# show plot with profit and losses:

performance = stat.to_pandas()["equity"]
make_plot_filled(performance.index, performance, name="PnL (equity)")

In [19]:
# show underwater chart:

UWchart = stat.to_pandas()["underwater"]
make_plot_filled(UWchart.index, UWchart, color="darkred", name="underwater chart")

In [20]:
# show rolling Sharpe ratio on a yearly basis:

SRchart = stat.to_pandas()["sharpe_ratio"].iloc[20:]
make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="rolling SR")